In [1]:
import matplotlib.pyplot as plt
import numpy as np
from neurolib.models.multimodel import MultiModel
from neurolib.models.multimodel.builder import HopfNetwork
from neurolib.utils.functions import getPowerSpectrum
from neurolib.utils.stimulus import ZeroInput

from hopf_cfc_network import SlowFastHopfNetwork
from neurolib.utils.parameterSpace import ParameterSpace
from neurolib.optimize.exploration import BoxSearch
from scipy.signal import find_peaks

import h5py
from cfcUtils import modulation_index_general, mean_vector_length, phase_locking_value, mutual_information


In [2]:
DURATION = 20.0 * 1000  # ms
DT = 0.1  # ms

def simulate(slow_to_fast_conn=0.,fast_to_slow_conn=.0):
    model = MultiModel(
        SlowFastHopfNetwork(
            number_of_slow_fast_units=1,
            slow_to_fast_connection=slow_to_fast_conn,
            fast_to_slow_connection=fast_to_slow_conn,
        )
    )
    model.params["duration"] = 20.0 * 1000
    model.params["sampling_dt"] = DT
    model.params["dt"] = DT

    model.run()

    return model

In [3]:
# run 50 times and calculate the CFC measures, then average them
mis = np.zeros(50)
mvls_angle = np.zeros(50)
mvls_abs = np.zeros(50)
plvs_angle = np.zeros(50)
plvs_abs = np.zeros(50)
minfos = np.zeros(50)

for i in range(50):
    model = simulate()
    freq_slow, pow_slow = getPowerSpectrum(model.x.T[:, 0], dt=0.1, maxfr=60, spectrum_windowsize=1)
    freq_fast, pow_fast = getPowerSpectrum(model.x.T[:, 1], dt=0.1, maxfr=60, spectrum_windowsize=1)

    peaks_fast, _ = find_peaks(pow_fast, height=max(1e-3, 1.0 * np.std(pow_fast)))
    peaks_slow, _ = find_peaks(pow_slow, height=max(1e-3, 0.5 * np.std(pow_slow)))

    phase_slow = np.arctan2(model.y[0, :], model.x[0, :])
    phase_fast = np.arctan2(model.y[1, :], model.x[1, :])
    amp_fast = np.sqrt(model.x[1, :] ** 2 + model.y[1, :] ** 2)

    mi = modulation_index_general(amp_fast, phase_slow, n_bins = 18)
    mvl = mean_vector_length(amp_fast, phase_slow)
    mvl_abs = np.absolute(mvl)
    mvl_angle = np.angle(mvl)
    plv = phase_locking_value(phase_fast, phase_slow)
    plv_abs = np.absolute(plv)
    plv_angle = np.angle(plv)
    minfo = mutual_information(phase_fast, phase_slow, bins=16, log2=False)

    mis[i] = mi
    mvls_angle[i] = mvl_angle
    mvls_abs[i] = mvl_abs
    plvs_angle[i] = plv_angle
    plvs_abs[i] = plv_abs
    minfos[i] = minfo

/home/lauraflyra/Documents/BCCN/Lab_Rotation_CFC/Hopf_CFC/Hopf_CFC_analysis/venv/lib/python3.8/site-packages/jitcdde/_jitcdde.py:1492: UserWarning: Your input past does not begin at t=0 but at t=0.1. Values before the beginning of the past will be extrapolated. You very likely do not want this.
  warn(f"Your input past does not begin at t=0 but at t={input[0].time}. Values before the beginning of the past will be extrapolated. You very likely do not want this.")
/tmp/tmppthwt18d/jitced.c: In function ‘get_past_anchors’:
/tmp/tmppthwt18d/jitced.c:176: warning: ignoring #pragma omp atomic [-Wunknown-pragmas]
  176 |  #pragma omp atomic capture
      | 
/tmp/tmppthwt18d/jitced.c:190: warning: ignoring #pragma omp critical [-Wunknown-pragmas]
  190 |   #pragma omp critical(pws)
      | 
/tmp/tmppthwt18d/jitced.c: In function ‘get_recent_state’:
/tmp/tmppthwt18d/jitced.c:285: warning: ignoring #pragma omp parallel [-Wunknown-pragmas]
  285 |  #pragma omp parallel for schedule(dynamic, 1)
  

Using default integration parameters.


  0%|          | 0/200000 [00:00<?, ?it/s]/home/lauraflyra/Documents/BCCN/Lab_Rotation_CFC/Hopf_CFC/Hopf_CFC_analysis/venv/lib/python3.8/site-packages/jitcdde/_jitcdde.py:792: UserWarning: The target time is smaller than the current time. No integration step will happen. The returned state will be extrapolated from the interpolating Hermite polynomial for the last integration step. You may see this because you try to integrate backwards in time, in which case you did something wrong. You may see this just because your sampling step is small, in which case there is no need to worry.
  warn("The target time is smaller than the current time. No integration step will happen. The returned state will be extrapolated from the interpolating Hermite polynomial for the last integration step. You may see this because you try to integrate backwards in time, in which case you did something wrong. You may see this just because your sampling step is small, in which case there is no need to worry.")
1

Using default integration parameters.


100%|██████████| 200000/200000 [00:00<00:00, 327631.50it/s]
/tmp/tmpyo4s1d3m/jitced_2.c: In function ‘get_past_anchors’:
/tmp/tmpyo4s1d3m/jitced_2.c:176: warning: ignoring #pragma omp atomic [-Wunknown-pragmas]
  176 |  #pragma omp atomic capture
      | 
/tmp/tmpyo4s1d3m/jitced_2.c:190: warning: ignoring #pragma omp critical [-Wunknown-pragmas]
  190 |   #pragma omp critical(pws)
      | 
/tmp/tmpyo4s1d3m/jitced_2.c: In function ‘get_recent_state’:
/tmp/tmpyo4s1d3m/jitced_2.c:285: warning: ignoring #pragma omp parallel [-Wunknown-pragmas]
  285 |  #pragma omp parallel for schedule(dynamic, 1)
      | 
In file included from /tmp/tmpyo4s1d3m/jitced_2.c:370:
/tmp/tmpyo4s1d3m/f.c: In function ‘eval_f’:
/tmp/tmpyo4s1d3m/f.c:1: warning: ignoring #pragma omp parallel [-Wunknown-pragmas]
    1 | #pragma omp parallel sections
      | 
/tmp/tmpyo4s1d3m/f.c:3: warning: ignoring #pragma omp section [-Wunknown-pragmas]
    3 | #pragma omp section
      | 
/tmp/tmpyo4s1d3m/f.c:5: warning: ignoring 

...

100%|██████████| 200000/200000 [00:00<00:00, 309591.40it/s]
/tmp/tmpem6y8amt/jitced_48.c: In function ‘get_past_anchors’:
/tmp/tmpem6y8amt/jitced_48.c:176: warning: ignoring #pragma omp atomic [-Wunknown-pragmas]
  176 |  #pragma omp atomic capture
      | 
/tmp/tmpem6y8amt/jitced_48.c:190: warning: ignoring #pragma omp critical [-Wunknown-pragmas]
  190 |   #pragma omp critical(pws)
      | 
/tmp/tmpem6y8amt/jitced_48.c: In function ‘get_recent_state’:
/tmp/tmpem6y8amt/jitced_48.c:285: warning: ignoring #pragma omp parallel [-Wunknown-pragmas]
  285 |  #pragma omp parallel for schedule(dynamic, 1)
      | 
In file included from /tmp/tmpem6y8amt/jitced_48.c:370:
/tmp/tmpem6y8amt/f.c: In function ‘eval_f’:
/tmp/tmpem6y8amt/f.c:1: warning: ignoring #pragma omp parallel [-Wunknown-pragmas]
    1 | #pragma omp parallel sections
      | 
/tmp/tmpem6y8amt/f.c:3: warning: ignoring #pragma omp section [-Wunknown-pragmas]
    3 | #pragma omp section
      | 
/tmp/tmpem6y8amt/f.c:5: warning: ign

Using default integration parameters.


100%|██████████| 200000/200000 [00:00<00:00, 297822.48it/s]
/tmp/tmpekvgjh71/jitced_49.c: In function ‘get_past_anchors’:
/tmp/tmpekvgjh71/jitced_49.c:176: warning: ignoring #pragma omp atomic [-Wunknown-pragmas]
  176 |  #pragma omp atomic capture
      | 
/tmp/tmpekvgjh71/jitced_49.c:190: warning: ignoring #pragma omp critical [-Wunknown-pragmas]
  190 |   #pragma omp critical(pws)
      | 
/tmp/tmpekvgjh71/jitced_49.c: In function ‘get_recent_state’:
/tmp/tmpekvgjh71/jitced_49.c:285: warning: ignoring #pragma omp parallel [-Wunknown-pragmas]
  285 |  #pragma omp parallel for schedule(dynamic, 1)
      | 
In file included from /tmp/tmpekvgjh71/jitced_49.c:370:
/tmp/tmpekvgjh71/f.c: In function ‘eval_f’:
/tmp/tmpekvgjh71/f.c:1: warning: ignoring #pragma omp parallel [-Wunknown-pragmas]
    1 | #pragma omp parallel sections
      | 
/tmp/tmpekvgjh71/f.c:3: warning: ignoring #pragma omp section [-Wunknown-pragmas]
    3 | #pragma omp section
      | 
/tmp/tmpekvgjh71/f.c:5: warning: ign

Using default integration parameters.


100%|██████████| 200000/200000 [00:00<00:00, 302691.62it/s]


In [7]:
print("Zero connection modulation index averaged: ",np.mean(mis))
print("Zero connection mean vector length averaged: absolute value",np.mean(mvls_abs), "angle: ", np.mean(mvls_angle))
print("Zero connection phase locking value averaged: absolute value",np.mean(plvs_abs), "angle: ", np.mean(plvs_angle))
print("Zero connection mutual information averaged: ",np.mean(minfos))


Zero connection modulation index averaged:  4.592010600458556e-09
Zero connection mean vector length averaged: absolute value 0.0010565758073750621 angle:  -0.3821953337960216
Zero connection phase locking value averaged: absolute value 0.0004924694462907957 angle:  0.1504671378472717
Zero connection mutual information averaged:  1.067440611357666
